# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import sys
from sql_queries import *
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=password")
cur = conn.cursor()
conn.set_session(autocommit=True)


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
#print(song_files)

In [5]:

filepath = song_files[0]
print(filepath)

/Users/theomar/data-engineering-udacity/Project1/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [6]:
df=pd.read_json(filepath,lines=True)
df = df.head()
print(df)


            artist_id  artist_latitude  artist_location  artist_longitude  \
0  AR7G5I41187FB4CE6C              NaN  London, England               NaN   

  artist_name   duration  num_songs             song_id            title  year  
0    Adam Ant  233.40363          1  SONHOTT12A8C13493C  Something Girls  1982  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id','year','duration']].values
song_data=list(song_data[0])
print(song_data)

['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print('value'+str(insert_table_queries))
record_to_insert = (song_data[0],song_data[1],song_data[2],song_data[3],song_data[4])
try:
    cur.execute(song_table_insert,record_to_insert )
except TypeError as error:
    e = sys.exc_info()[0]
    print(error)
                
conn.commit()

value['INSERT INTO songs(\nsong_id,\ntitle,\nartist_id,\nyear,\nduration\n)VALUES(%s,%s,%s,%s,%s);\n']


UniqueViolation: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SONHOTT12A8C13493C) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location','artist_latitude','artist_longitude']].values
artist_data = list(artist_data[0])
print(artist_data)

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
artist_data_to_insert=(artist_data[0],artist_data[1],artist_data[2],artist_data[3],artist_data[4])
cur.execute(artist_table_insert, artist_data_to_insert)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(AR7G5I41187FB4CE6C) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]
print(filepath)

/Users/theomar/data-engineering-udacity/Project1/data/log_data/2018/11/2018-11-11-events.json


In [13]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
def my_filter_callback_by_NextSong(row):
    return row.page=='NextSong'


df = df[df.apply(my_filter_callback_by_NextSong, axis=1)]
#converting ts value to date time
df['date_time']=(pd.to_datetime(df['ts'],unit='ms')) 

In [15]:
time_data=[[]]
from datetime import datetime

    

In [16]:
t = df[['date_time','ts']]
t.head()




,date_time,ts
0,2018-11-11 02:33:56.796,1541903636796
1,2018-11-11 02:36:10.796,1541903770796
2,2018-11-11 02:40:34.796,1541904034796
4,2018-11-11 04:36:13.796,1541910973796
5,2018-11-11 04:36:46.796,1541911006796


In [17]:

import time
for index, row in t.iterrows():
    data=[datetime.fromtimestamp(t['ts'][index]/1000).strftime('%Y-%m-%d %H:%M:%S.%f'),
          int(t['date_time'][index].hour),
          int(t['date_time'][index].day),
          int(t['date_time'][index].week),
          int(t['date_time'][index].month),
          int(t['date_time'][index].year),
          int(t['date_time'][index].dayofweek)
        ]
    time_data.append(data)



In [18]:
time_data = (time_data)
column_labels = ('timestamp','hour','day','week','month','year','weekday')

In [19]:
time_df = pd.DataFrame(time_data,columns=column_labels)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,None,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-11-10 20:33:56.796000,2.0,11.0,45.0,11.0,2018.0,6.0
2,2018-11-10 20:36:10.796000,2.0,11.0,45.0,11.0,2018.0,6.0
3,2018-11-10 20:40:34.796000,2.0,11.0,45.0,11.0,2018.0,6.0
4,2018-11-10 22:36:13.796000,4.0,11.0,45.0,11.0,2018.0,6.0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    if i!=0:
        print(list(row))
        cur.execute(time_table_insert, list(row))
        conn.commit()

['2018-11-10 20:33:56.796000', 2.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-10 20:36:10.796000', 2.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-10 20:40:34.796000', 2.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-10 22:36:13.796000', 4.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-10 22:36:46.796000', 4.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 03:56:28.796000', 9.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:00:47.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:04:07.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:08:28.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:12:00.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:27:43.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:33:44.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:36:22.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:40:14.796000', 10.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
['2018-11-11 04:43:23.796000', 10.0, 11.0, 45.0, 11.0,

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.